# Модуль 4.7 — RAG по реальным Markdown (real_rag_folder)

**Цель:** показать, как лучше парсить реальные `.md` файлы с кодовыми блоками и примерами селекторов.

**Что сделаем:**
- разберём структуру и цели предобработки
- распарсим Markdown по заголовкам (без потери кода)
- соберём документы с метаданными
- построим RAG и зададим вопрос

In [ ]:
%pip -q install -U \
  langchain \
  langchain-community \
  langchain-openai \
  chromadb \
  pydantic==2.12.3 \
  requests==2.32.4

## Если Colab ругается на зависимости

Иногда в Colab установлены библиотеки с жёсткими версиями (например, `google-adk` и `opentelemetry`).
Если после установки появляются конфликты — зафиксируйте совместимые версии ниже.

In [ ]:
%pip -q install -U \
  opentelemetry-api==1.37.0 \
  opentelemetry-sdk==1.37.0 \
  opentelemetry-proto==1.37.0 \
  opentelemetry-exporter-otlp-proto-common==1.37.0 \
  opentelemetry-exporter-otlp-proto-grpc==1.37.0

# На случай, если в окружении уже стоит более новая версия
def _force_pins():
    import sys, subprocess
    pkgs = [
        "opentelemetry-api==1.37.0",
        "opentelemetry-sdk==1.37.0",
        "opentelemetry-proto==1.37.0",
        "opentelemetry-exporter-otlp-proto-common==1.37.0",
        "opentelemetry-exporter-otlp-proto-grpc==1.37.0",
    ]
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "-U", "--force-reinstall"] + pkgs)

_force_pins()

## Настройка ключа и base URL

Для AITunnel укажите `OPENAI_API_KEY` и `OPENAI_BASE_URL`. В Colab лучше хранить ключ в переменной окружения.

In [ ]:
import os
from getpass import getpass
from dotenv import load_dotenv

load_dotenv()

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass("Введите OPENAI_API_KEY: ")

if not os.environ.get("OPENAI_BASE_URL"):
    os.environ["OPENAI_BASE_URL"] = "https://api.aitunnel.ru/v1/"

## Почему здесь нужен особый парсинг

В ваших файлах много:
- больших кодовых блоков (`html`, `css`, `xpath`)
- повторяющейся структуры «пример → HTML → селекторы»
- заголовков `##` / `###`, которые лучше использовать как границы чанков

Поэтому мы:
1) **Не удаляем кодовые блоки** (они несут ключевые ответы)
2) **Режем по заголовкам**, а не по длине
3) **Сохраняем метаданные** (файл, название примера)

## Парсинг Markdown по заголовкам

Мы разбиваем текст на секции по `##` / `###` и превращаем каждую секцию в отдельный документ.

In [ ]:
from pathlib import Path
import re
from langchain_core.documents import Document

SOURCE_DIR = Path("./real_rag_folder")

heading_re = re.compile(r"^(###+)\s+(.*)")


def split_by_headings(text: str):
    sections = []
    current_title = "Без заголовка"
    current_lines = []

    for line in text.splitlines():
        m = heading_re.match(line.strip())
        if m:
            # сохраняем предыдущую секцию
            if current_lines:
                sections.append((current_title, "\n".join(current_lines).strip()))
            current_title = m.group(2).strip()
            current_lines = []
        else:
            current_lines.append(line)

    if current_lines:
        sections.append((current_title, "\n".join(current_lines).strip()))

    return sections


def normalize_text(text: str) -> str:
    # Сохраняем кодовые блоки, но убираем лишние пробелы вокруг
    text = re.sub(r"\n{3,}", "\n\n", text)
    return text.strip()


all_docs = []
for md_path in sorted(SOURCE_DIR.glob("*.md")):
    raw = md_path.read_text(encoding="utf-8", errors="ignore")
    for title, content in split_by_headings(raw):
        content = normalize_text(content)
        if len(content) < 50:
            continue
        all_docs.append(
            Document(
                page_content=f"### {title}\n\n{content}",
                metadata={
                    "source": md_path.name,
                    "section": title,
                },
            )
        )

print("Секций найдено:", len(all_docs))
print("Пример секции:\n", all_docs[0].page_content[:500])

## RAG по реальным секциям

Строим векторную базу и задаём вопрос по селекторам.

In [ ]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import Chroma
from langchain.storage import LocalFileStore
from langchain.embeddings import CacheBackedEmbeddings
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

store = LocalFileStore("./cache/embeddings")
embeddings = OpenAIEmbeddings(model="text-embedding-3-small", base_url=os.environ.get("OPENAI_BASE_URL"))
cached_embedder = CacheBackedEmbeddings.from_bytes_store(embeddings, store, namespace="openai")

vectorstore = Chroma.from_documents(
    documents=all_docs,
    embedding=cached_embedder,
    persist_directory="./db/chroma_real_md",
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

llm = ChatOpenAI(
    model="gpt-5.2-chat",
    temperature=0.2,
    max_tokens=256,
    base_url=os.environ.get("OPENAI_BASE_URL"),
)

class SelectorResult(BaseModel):
    selector_type: str = Field(description="Тип селектора: css | xpath | playwright | bs4")
    selectors: list[str] = Field(description="Список подходящих селекторов")
    explanation: str = Field(description="Короткое объяснение выбора")

parser = PydanticOutputParser(pydantic_object=SelectorResult)

prompt = PromptTemplate(
    template=(
        "Ты эксперт по селекторам. Используй КОНТЕКСТ ниже.\n"
        "Верни ответ СТРОГО в JSON по инструкции.\n"
        "{format_instructions}\n\n"
        "КОНТЕКСТ:\n{context}\n\n"
        "HTML:\n{html}\n\n"
        "Вопрос: {question}"
    ),
    input_variables=["context", "html", "question"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# Пример HTML
html_snippet = """
<nav id="main-nav" class="navigation primary">
  <ul class="menu">
    <li class="menu-item active"><a href="/">Главная</a></li>
    <li class="menu-item"><a href="/about" data-section="info">О нас</a></li>
  </ul>
</nav>
""".strip()

question = "Подбери селектор для активного пункта меню."

retrieved = retriever.get_relevant_documents(question)
context = "\n\n".join([d.page_content for d in retrieved])

filled = prompt.format(context=context, html=html_snippet, question=question)
raw = llm.invoke(filled)

# Парсинг в структурированный JSON
result = parser.parse(raw.content)
print(result.model_dump())

## Практические рекомендации

- Не удаляйте кодовые блоки — в них главная ценность для RAG.
- Режьте по заголовкам `##`/`###`, это соответствует структуре примеров.
- Добавляйте метаданные (файл, пример, тема) — это помогает фильтрации.
- Для крупных файлов можно разбивать ещё по подзаголовкам внутри примера.